<a href="https://colab.research.google.com/github/fabiodr/colabs/blob/main/clock_hands_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is the angle between the clock hands at a certain time?

The tweet that inspired it all: https://x.com/karpathy/status/1961128638725923119

In [ ]:
!pip install dspy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 16.6 MB/s eta 0:00:00
  Created wheel for madoka: filename=madoka-0.7.1-cp312-cp312-linux_x86_64.whl size=779301 sha256=7d21a2245c4fa90286313c6b3aa18d6a23cde1f613a9051dd654c02c2afb2dcb
  Stored in directory: /root/.cache/pip/wheels/1b/20/03/40ecb5c612b2075d15f0afb855bf59c090d708336292458281
Successfully built madoka


In [ ]:
import polars as pl
import dspy

First lets load the dataset from huggingface, we can do it directly through a url.

In [ ]:
df = pl.read_csv('hf://datasets/rasmus1610/clock_hand_angles/clock_hand_angles.csv')
df.head()


time,angle
str,i64
"""00:00""",0
"""00:01""",5
"""00:02""",11
"""00:03""",16
"""00:04""",22


Now we will define our main LLM, gemma-3n-E4B by Google. A really tiny LLM that can be easily run on edge devices.

In [ ]:
from google.colab import userdata

lm = dspy.LM("together_ai/google/gemma-3n-E4B-it", api_key=userdata.get('TOGETHER_API_KEY'), max_tokens=8000)
lm("Hello world")

["Hello there! 👋 \n\nIt's great to be here. How can I help you today? 😊 \n\nJust let me know what you need - I can try my best to answer questions, generate text, translate languages, write different kinds of creative content, and more! \n"]

In [ ]:
dspy.configure(lm=lm)

Turn the dataset into a list of examples

In [ ]:
examples = [dspy.Example({"time": time, "angle": angle}).with_inputs("time") for time, angle in df.iter_rows()]


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(examples, test_size=0.8, random_state=42)

Now lets define our DSPy module. We'll stick with a simple `ChainOfThought` module.

In [ ]:
get_angle = dspy.ChainOfThought("time -> clock_hands_angle: int")

In [ ]:
def match(gold, pred, trace=None):
    return gold.angle == pred.clock_hands_angle


In [ ]:
ev = dspy.Evaluate(devset=test, display_progress=True, metric=match, num_threads=16)

In [ ]:
ev(get_angle)

2025/09/21 20:09:57 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.
2025/09/21 20:09:57 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


Average Metric: 2.00 / 3 (66.7%):   0%|          | 2/624 [00:00<00:17, 36.07it/s]

2025/09/21 20:09:57 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


Average Metric: 2.00 / 4 (50.0%):   0%|          | 3/624 [00:00<00:19, 32.08it/s]

2025/09/21 20:09:57 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


...

2025/09/21 20:09:58 ERROR dspy.utils.parallelizer: Error for Example({'time': '03:33', 'angle': 91}) (input_keys={'time'}): Adapter JSONAdapter failed to parse the LM response. 

LM Response: ```json
{
  "reasoning": "At 3:33, the hour hand is a little past the 3 and the minute hand is on the 6. The hour hand moves 360 degrees in 12 hours, or 30 degrees per hour. It also moves 0.5 degrees per minute. The minute hand moves 360 degrees in 60 minutes, or 6 degrees per minute. \n\nHour hand position: 3 * 30 + 33 * 0.5 = 90 + 16.5 = 106.5 degrees\nMinute hand position: 6 * 6 = 36 degrees\n\nThe angle between the hands is |106.5 - 36| = 70.5 degrees. Since we want the smaller angle, the angle is 70.5 degrees. However, the question asks for the clock hands angle, which is the smaller angle between the two hands. The hour hand is at 3 and the minute hand is at 6. The angle between 3 and 6 is 3/12 of a full circle, which is 3 * 30 = 90 degrees. The difference is 90 - 70.5 = 19.5 degrees. The ot

Average Metric: 19.00 / 27 (70.4%):   4%|▍         | 27/624 [00:01<00:30, 19.83it/s]

2025/09/21 20:09:58 ERROR dspy.utils.parallelizer: Error for Example({'time': '11:35', 'angle': 137}) (input_keys={'time'}): Adapter JSONAdapter failed to parse the LM response. 

LM Response: ```json
{
  "reasoning": "At 11:35, the hour hand is past the 11 and moving towards the 12. The minute hand is on the 7. The angle between each number on the clock is 30 degrees (360/12). The hour hand is 5/12 of the way around the clock from the 12, and the minute hand is 7/12 of the way around the clock from the 12. Therefore, the angle between the hands is |(5/12 * 30) - (7/12 * 30)| = |12.5 - 17.5| = 5 degrees. However, the question asks for the angle between the hands, which is 5 * 2 = 10 degrees. The hour hand is at 11 and the minute hand is at 7. The angle between 11 and 7 is 4 hours and 60 minutes, which is 4.5 hours. The angle is 4.5 * 30 = 135 degrees. The minute hand is at 35 minutes, which is 35/60 of the way around the clock. So the minute hand is at 35/60 * 360 = 210 degrees. The ho

2025/09/21 20:09:58 WARNING dspy.utils.parallelizer: Execution cancelled due to errors or interruption.
2025/09/21 20:09:58 ERROR dspy.utils.parallelizer: Error for Example({'time': '02:13', 'angle': 11}) (input_keys={'time'}): Adapter JSONAdapter failed to parse the LM response. 

LM Response: ```json
{
  "reasoning": "At 2:13, the hour hand is past the 2 and a bit into the 3. The minute hand is on the 1. The hour hand moves 360 degrees / 12 hours = 30 degrees per hour. It also moves 30 degrees / 60 minutes = 0.5 degrees per minute. So, at 2:13, the hour hand is at 2 * 30 + 13 * 0.5 = 60 + 6.5 = 66.5 degrees. The minute hand moves 360 degrees / 60 minutes = 6 degrees per minute. So, at 13 minutes, the minute hand is at 13 * 6 = 78 degrees. The difference between the two hands is |78 - 66.5| = 11.5 degrees. However, the question asks for the angle between the clock hands, which is the smaller angle. The other angle is 360 - 11.5 = 348.5 degrees. The smaller angle is 11.5 degrees. Since

Exception: Execution cancelled due to errors or interruption.

The evaluation of the unoptimized module won't even finish due to token limits. The LLM is just brabbling along. Let's see what GEPA can do here.

Let's embellish our metric a little bit to also return a free-text feedback.

In [ ]:
def gepa_match(gold, pred, trace=None, pred_name=None, pred_trace=None):
    score = match(gold, pred)
    if score == False:
        feedback = f"The angle is wrong. The correct angle is: {gold.angle} degrees (You calculated {pred.clock_hands_angle})"
        return dspy.Prediction(score=0, feedback=feedback)
    return dspy.Prediction(score=1, feedback=f"You calculated the correct angle of {pred.clock_hands_angle}")




Now lets use GEPA to optimize our module for this task. We'll use GPT-5 as the reflection LLM (the one that comes up with the better instructions) and lets set a maximum of 200 metric calls (rollouts)

In [ ]:
dspy.teleprompt.GEPA?

In [ ]:
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [ ]:
from dspy.teleprompt import GEPA

teleprompt = GEPA(
    metric=gepa_match,
    max_full_evals=1,
    num_threads=16,
    reflection_lm=dspy.LM(model="gpt-5", temperature=1.0, max_tokens=32000, api_key=userdata.get('OPENAI_API_KEY'))
)

In [ ]:
opt = teleprompt.compile(get_angle, trainset=train, valset=val)

2025/09/22 12:30:18 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 156 metric calls of the program. This amounts to 1.00 full evals on the train+val set.
2025/09/22 12:30:18 INFO dspy.teleprompt.gepa.gepa: Using 32 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
GEPA Optimization:   0%|          | 0/156 [00:00<?, ?rollouts/s]2025/09/22 12:33:01 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=8000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.
2025/09/22 12:33:01 ERROR dspy.utils.parallelizer: Error for Example({'time': '07:07', 'angle': 171}) (input_keys={'time'}): Adapter JSONAdapter failed to parse the LM response.

Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:12<00:00,  4.22s/it]

2025/09/22 12:34:58 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)


2025/09/22 12:35:28 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: Task
- Input: a single field time formatted as "HH:MM" (00–23 for hours is allowed; treat the clock as 12-hour analog).
- Output: produce only the field clock_hands_angle with the smaller angle (in degrees) between the hour and minute hands at that time.

How to compute
1) Parse HH and MM as integers.
2) Convert the hour to 12-hour format: hour12 = HH % 12.
3) Minute hand angle from 12 o’clock: minute_angle = 6 * MM  (since 360°/60 = 6° per minute).
4) Hour hand angle from 12 o’clock: hour_angle = 30 * hour12 + 0.5 * MM  (since 30° per hour + 0.5° per minute).
5) Raw difference: diff = abs(hour_angle - minute_angle).
6) Smaller angle: angle = min(diff, 360 - diff). This will be in [0, 180].
7) Output angle as an integer by truncating any fractional part (e.g., 132.5 -> 132). Do not round to nearest; do not include decimals or units.

Output format rules
- Return only the required field clock

Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:03<00:00,  1.26s/it]

...


Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:02<00:00,  1.05it/s]

2025/09/22 12:37:27 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)


2025/09/22 12:37:27 INFO dspy.teleprompt.gepa.gepa: Iteration 29: All subsample scores perfect. Skipping.
2025/09/22 12:37:27 INFO dspy.teleprompt.gepa.gepa: Iteration 29: Reflective mutation did not propose a new candidate
GEPA Optimization:  99%|█████████▊| 154/156 [07:08<00:02,  1.16s/rollouts]2025/09/22 12:37:27 INFO dspy.teleprompt.gepa.gepa: Iteration 30: Selected program 1 score: 1.0


Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:03<00:00,  1.13s/it]

2025/09/22 12:37:30 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2025/09/22 12:37:30 INFO dspy.teleprompt.gepa.gepa: Iteration 30: All subsample scores perfect. Skipping.
2025/09/22 12:37:30 INFO dspy.teleprompt.gepa.gepa: Iteration 30: Reflective mutation did not propose a new candidate
GEPA Optimization:  99%|█████████▊| 154/156 [07:12<00:05,  2.81s/rollouts]

In [ ]:
ev(opt)

Average Metric: 621.00 / 624 (99.5%): 100%|██████████| 624/624 [02:35<00:00,  4.00it/s]

2025/09/22 12:40:48 INFO dspy.evaluate.evaluate: Average Metric: 621 / 624 (99.5%)


EvaluationResult(score=99.52, results=<list of 624 results>)

Lets look at the prompt GEPA came up with.

In [ ]:
opt.inspect_history()





[2025-09-22T12:40:48.122223]

System message:

Your input fields are:
1. `time` (str):
Your output fields are:
1. `reasoning` (str): 
2. `clock_hands_angle` (int):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## time ## ]]
{time}

[[ ## reasoning ## ]]
{reasoning}

[[ ## clock_hands_angle ## ]]
{clock_hands_angle}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Task
        - Input: a single field time formatted as "HH:MM" (00–23 for hours is allowed; treat the clock as 12-hour analog).
        - Output: produce only the field clock_hands_angle with the smaller angle (in degrees) between the hour and minute hands at that time.
        
        How to compute
        1) Parse HH and MM as integers.
        2) Convert the hour to 12-hour format: hour12 = HH % 12.
        3) Minute hand angle from 12 o’clock: minute_angle = 6 * MM  (

Interesting how the reasoning trace is basically just code. Very efficient. Before it was so long that we went over the token limit.